# RAG Demo (Gemini + Google Embeddings + FAISS)

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

load_dotenv()
DATA_DIR = os.getenv('DATA_DIR', '..\\RAG Project Dataset')
INDEX_DIR = os.getenv('INDEX_DIR', '.\\faiss_index')

PROMPT = """
You are a research assistant.
Answer ONLY using the provided context.
If the answer is not present, say:
"I could not find sufficient information in the documents."

Context:
{context}

Question:
{question}

Answer:
"""


In [ ]:
def load_pdfs(data_dir):
    pdf_paths = sorted(Path(data_dir).rglob('*.pdf'))
    docs = []
    for p in pdf_paths:
        docs.extend(PyPDFLoader(str(p)).load())
    return docs

docs = load_pdfs(DATA_DIR)
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,
    chunk_overlap=150,
)
chunks = splitter.split_documents(docs)

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vectorstore = FAISS.from_documents(chunks, embeddings)


In [ ]:
prompt = PromptTemplate(
    template=PROMPT,
    input_variables=['context', 'question'],
)
llm = ChatGoogleGenerativeAI(model='gemini-pro', temperature=0)
retriever = vectorstore.as_retriever(search_kwargs={'k': 4})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type='stuff',
    return_source_documents=True,
    chain_type_kwargs={'prompt': prompt},
)

result = qa('What are the two sub-layers in each Transformer encoder layer?')
answer = result['result']
sources = [
    {'document': doc.metadata.get('source'), 'page': doc.metadata.get('page')}
    for doc in result['source_documents']
]

print('Answer:', answer)
print('Sources:')
for s in sources:
    name = os.path.basename(s['document'] or '')
    page = s['page']
    if page is not None:
        print(f'- {name} (Page {page})')
    else:
        print(f'- {name}')
